## Classification and Regression Tree (from scratch)

##### This implementation was referenced from this [post](https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/).

In [1]:
import pandas as pd
import numpy as np

In [4]:
def gini_index(groups, classes):
    '''
    Returns the gini index for a particular split which gives the group distribution as groups.
    Classes are labels like [-1, 1] or [0, 1] or ['yes', 'no']
    '''
    # count all samples at the split point (all points in all groups)
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group)) # group size
        if size == 0:
            continue # Don't divide by zero :p
        score = 0.0
        for _class_val in classes:
            p = [row[-1] for row in group].count(_class_val) / size
            score += p * p
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
        
    return gini

In [6]:
# Sanity check
print(gini_index([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1])) # 50/50 split
print(gini_index([[[1, 0], [1, 0]], [[1, 1], [1, 1]]], [0, 1])) # Perfect split

0.5
0.0


In [8]:
# To define a split you need two parts of information.
# What value should split the dataset? (numerical value)
# Whose value should split the dataest? (Index of the attribute)

def _split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] >= value:
            right.append(row)
        else:
            left.append(row)
            
    return left, right

In [9]:
# We need a exhaustive and greedy search for the best set of index and value pair that gives best split

def get_split(dataset):
    '''
    Get the best split from the given dataset by using a greedy approach
    '''
    # Get a list of class labels
    _labels = list(set([row[-1] for row in dataset]))
    _index, _value, _score, _groups = 999, 999, 999, None
    for index in range(len(dataset[0]) - 1):
        for row in dataset:
            groups = _split(index, row[index], dataset)
            gini = gini_index(groups, _labels)
            if gini < _score:
                _index = index
                _value = row[index]
                _groups = groups
                _score = gini
                
    return {'index': _index, 'value': _value, 'groups': _groups}

In [10]:
# Now that we can generate a split from a dataset, let's apply this recursively to generate a binary tree.

In [15]:
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

def split(node, max_depth, min_size, depth):
    '''
    Recursively splits and creates a tree
    '''
    left, right = node['groups']
    del node['groups']
    
    # Check if one of the groups is empty
    if len(left) == 0 or len(right) == 0:
        node['left'] = node['right'] = to_terminal(left + right)
        return 
        
    # Check if maximum depth exceeded
    if depth > max_depth:
        node['left'] = node['right'] = to_terminal(left + right)
        return 
    
    # Process left child
    if len(left) < min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth + 1)
       
    # Process right child
    if len(right) < min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth + 1)

In [16]:
# Build a decision tree
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

In [17]:
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']